# Mnist dataset building vanilla auto-encoder using three fully-connected layers

In [2]:
# import the needed packages
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_squared_error

In [3]:
# read the data
(X_train_full, _), (X_test, _) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# split, reshape and normalize the training, validation and test sets
X_train = X_train_full[:-10000].reshape(-1, 784)/255.
X_valid = X_train_full[-10000:].reshape(-1, 784)/255.
X_test = X_test.reshape(-1, 784)/255.

In [5]:
# define a function that creates a vanilla autoencoder with a specified size for the latent space
def built_model(encoding_dim):
    # fix the random numbers generator (it would be better to fix also the random.seed and np.random.seed)
    tf.random.set_seed(42)

    # define the input size
    input_img = Input(shape = (784,))

    # "encoded" is the encoded representation of the input
    encoded = Dense(encoding_dim, activation = 'relu')(input_img)

    # "decoded" is the lossy reconstruction of the input
    decoded = Dense(784, activation = 'sigmoid')(encoded)

    # this model maps an input to its reconstruction
    model = Model(input_img, decoded)

    return model

In [6]:
latent_space_sizes = [10, 100, 250]

models = {}
for latent_size in latent_space_sizes:

    print(f"\n\nTraining with latent space size: {latent_size}\n\n")
    # create the model
    models[latent_size] = built_model(latent_size)

    # visualize the model
    plot_model(models[latent_size], show_shapes = True)

    # compile the model
    models[latent_size].compile(optimizer = "adam", loss = "binary_crossentropy")

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                      patience=5,
                                                      restore_best_weights=True)

    history = models[latent_size].fit(X_train, X_train, epochs=100, batch_size = 32,
                                      validation_data=(X_valid, X_valid),
                                      callbacks=[early_stopping])

    reconstructed = models[latent_size].predict(X_test)
    mse = mean_squared_error(X_test, reconstructed)

    print(f"\n\nLatent space size: {latent_size:3d}, {mse:.6f}\n\n")



Training with latent space size: 10


Epoch 1/100
1563/1563 [==============================] - 6s 3ms/step - loss: 0.2177 - val_loss: 0.1704
Epoch 2/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1672 - val_loss: 0.1632
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1621 - val_loss: 0.1599
Epoch 4/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1591 - val_loss: 0.1574
Epoch 5/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1573 - val_loss: 0.1564
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1563 - val_loss: 0.1556
Epoch 7/100
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1557 - val_loss: 0.1550
Epoch 8/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1552 - val_loss: 0.1547
Epoch 9/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1548 - val_loss: 0.1543
Epoch 10/100
1563/1563 [=========